In [4]:
import json
import math
import queue as que

with open("./dict_files/Coord.json") as cord:
    coords = json.load(cord)
with open("./dict_files/Cost.json") as Cost:
    cost = json.load(Cost)
with open("./dict_files/Dist.json") as Dist:
    dist = json.load(Dist)
with open("./dict_files/G.json") as G:
    graph = json.load(G)

# Returns euclidean distance between the two nodes that are passed as arguments to this function
def euc_distance(node, connected_node):
    node_x, node_y = coords[node]
    connected_node_x, connected_node_y = coords[connected_node]
    return math.sqrt((node_x - connected_node_x) ** 2 + (node_y - connected_node_y) ** 2)

# This function manages a dictionary for all the visited nodes' paths
# If any node is visited then it's path is saved as an object in the dictionary, and this
# function will return the cost to reach that node along with the entire path

def append_to_path(path, node, parent):
    # Base case, if the node is empty then only one node is appended to the path and cost 0 is returned
    # as their is only one node in the path
    if len(path) == 0 and parent is None:
        return [node], 0

    # If the path already has one or more nodes in it, then the current node is appended into the parent's path
    # and the resulting path is allocated to the current node. The accumulated cost to reach the current node
    # is also returned
    path_to_return = []
    if parent in path:
        travel_cost = 0
        index = 0
        for path_node in path:
            # If we reach the parent node in path, then we append current node to it and the resulting path to the
            # current node is returned along with it's accumulated cost
            if path_node == parent:
                path_to_return.append(parent)
                travel_cost += cost[path[index]+","+node]
                path_to_return.append(node)
                return path_to_return, travel_cost
            path_to_return.append(path_node)
            travel_cost += cost[(path[index]+","+path[index + 1])]
            index += 1

# This function displays the path as specified in the requirements of any path_list that is passed to it as argument
def display_path(path_list):
    for i in range(len(path_list)):
        if i == len(path_list) - 1:
            print(path_list[i])
            break
        print(path_list[i]+" -> ", end="")

# Implementation of Greedy Best First Search for task1
def GBFS(start_node, goal_node):
    # PriorityQueue is maintained for sorting the accumulated cost for the current node
    priorityQueue = que.PriorityQueue()
    priorityQueue.put((0, start_node, None))
    visited_nodes = []
    # Total path is a dictionary that maintains the path_list for all the visited nodes
    total_path = {"1": ["1"]}
    total_cost = 0

    while not priorityQueue.empty():

        euc_dist, current_node, parent_node = priorityQueue.get()

        # In exception to the first iteration, the current node is appended to the path and resulting cost is
        # provided
        if parent_node is not None:
            path_list = total_path[parent_node]
            path_list, total_cost = append_to_path(path_list, current_node, parent_node)
            total_path[current_node] = path_list

        if current_node not in visited_nodes:
            visited_nodes.append(current_node)
            if current_node == goal_node:
                print("Shortest path: ", end="")
                display_path(total_path[current_node])
                print("Shortest distance: ", calculate_dist(total_path[current_node]))
                print("Total energy cost: ", calculate_cost(total_path[current_node]))
                return
            neighbors = graph[current_node]

            for neighbor in neighbors:
                # For GBFS, the approximate euc_dist to reach the goal_node from neighbor is stored in the priority
                # queue and thus based on shortest euc_dist, the algo works
                neighbor_euc_dist = euc_distance(neighbor, goal_node)
                priorityQueue.put((neighbor_euc_dist, neighbor, current_node))

# This function simply calculates the entire accumulated energy cost for any path list provided to it
def calculate_cost(path):
    path_cost = 0
    for index in range(len(path) - 1):
        path_cost += cost[(path[index] + "," + path[index + 1])]
    return path_cost

# This function simply calculates the entire accumulated distance for any path list provided to it
def calculate_dist(path):
    path_cost = 0
    for index in range(len(path) - 1):
        path_cost += dist[(path[index] + "," + path[index + 1])]
    return path_cost

# For task 2, implementation of Uniform Cost Search
def UCS(start_node, goal_node, energy_budget = 287932):
    # PriorityQueue is maintained for sorting the accumulated cost for the current node
    priorityQueue = que.PriorityQueue()
    priorityQueue.put((0, start_node, None))
    visited_nodes = []
    # Total path is a dictionary that maintains the path_list for all the visited nodes
    total_path = {"1": ["1"]}
    total_cost = 0

    while not priorityQueue.empty():
        travel_cost, current_node, parent_node = priorityQueue.get()

        # In exception to the first iteration, the current node is appended to the path and resulting cost is
        # provided
        if parent_node is not None:
            path_list = total_path[parent_node]
            path_list, total_cost = append_to_path(path_list, current_node, parent_node)
            total_path[current_node] = path_list

        visited_nodes.append(current_node)

        if current_node == goal_node:
            if calculate_cost(total_path[current_node]) > energy_budget:
                visited_nodes.remove(current_node)
            else:
                print("Shortest path: ", end="")
                display_path(total_path[current_node])
                print("Shortest distance: ", calculate_dist(total_path[current_node]))
                print("Total energy cost: ", calculate_cost(total_path[current_node]))
                return

        neighbors = graph[current_node]

        for neighbor in neighbors:
            # For UCS, the accumulated cost from start_node till neighbor node is stored in the priority
            # queue and thus based on shortest euc_dist, the algo works
            if neighbor not in visited_nodes or neighbor == goal_node:
                neighbor_dist = cost[current_node+","+neighbor]
                priorityQueue.put((neighbor_dist + travel_cost, neighbor, current_node))

def Astar(start_node, goal_node, energy_budget = 287932):
    # PriorityQueue is maintained for sorting the accumulated cost for the current node
    priorityQueue = que.PriorityQueue()
    priorityQueue.put((0, start_node, None))
    visited_nodes = []
    # Total path is a dictionary that maintains the path_list for all the visited nodes
    total_path = {"1": ["1"]}
    total_cost = 0

    while not priorityQueue.empty():

        travel_cost, current_node, parent_node = priorityQueue.get()
        dist_to_reach_current_node = 0

        # In exception to the first iteration, the current node is appended to the path and resulting cost is
        # provided
        if parent_node is not None:
            path_list = total_path[parent_node]
            path_list, dist_to_reach_current_node = append_to_path(path_list, current_node, parent_node)
            total_path[current_node] = path_list

        if current_node not in visited_nodes:
            visited_nodes.append(current_node)
            if current_node == goal_node:
                if calculate_cost(total_path[current_node]) > energy_budget:
                    visited_nodes.remove(current_node)
                else:
                    print("Shortest path: ", end="")
                    display_path(total_path[current_node])
                    print("Shortest distance: ", calculate_dist(total_path[current_node]))
                    print("Total energy cost: ", calculate_cost(total_path[current_node]))
                    return

            neighbors = graph[current_node]

            for neighbor in neighbors:
                # For Astar, the accumulated cost from start_node till neighbor node along with the estimated euc_dist
                # to reach the goal node from the neighbor is stored in the priority queue and thus based on shortest
                # euc_dist, the algo works
                neighbor_dist = cost[current_node+","+neighbor]
                dist_to_reach_neighbor = dist_to_reach_current_node + neighbor_dist
                euc_dist_to_goal_node = euc_distance(neighbor, goal_node)
                priorityQueue.put((dist_to_reach_neighbor + euc_dist_to_goal_node, neighbor, current_node))



In [8]:
if __name__ == '__main__':

    print("Task 1: Solving relaxed version of NYC instance with Greedy Best-First Search:")
    GBFS("1", "50")
    print("\n")
    print("Task 2: Solving NYC instance with UCS:")
    UCS("1", "50", 287932)
    print("\n")
    print("Task 3: Solving NYC instance with A* search:")
    Astar("1", "50", 287932)

Task 1: Solving relaxed version of NYC instance with Greedy Best-First Search:
Shortest path: 1 -> 1363 -> 1358 -> 1357 -> 1356 -> 1276 -> 1273 -> 1277 -> 1269 -> 1241 -> 1240 -> 1235 -> 956 -> 953 -> 955 -> 947 -> 944 -> 948 -> 949 -> 952 -> 1000 -> 998 -> 994 -> 995 -> 996 -> 987 -> 986 -> 979 -> 980 -> 969 -> 977 -> 881 -> 887 -> 885 -> 889 -> 2365 -> 2364 -> 879 -> 878 -> 877 -> 865 -> 862 -> 863 -> 860 -> 490 -> 489 -> 486 -> 488 -> 2019 -> 2022 -> 2000 -> 1996 -> 1998 -> 1999 -> 2005 -> 2007 -> 2009 -> 2011 -> 2059 -> 2061 -> 2062 -> 2065 -> 2057 -> 2050 -> 2051 -> 1979 -> 1975 -> 1967 -> 1966 -> 1974 -> 1973 -> 1971 -> 1970 -> 1948 -> 1937 -> 1939 -> 1935 -> 1931 -> 1934 -> 1673 -> 1675 -> 1674 -> 1837 -> 1834 -> 1829 -> 1818 -> 1816 -> 1815 -> 1634 -> 1814 -> 1755 -> 1754 -> 1753 -> 1749 -> 1752 -> 1748 -> 1746 -> 1732 -> 1695 -> 1694 -> 1701 -> 1702 -> 1699 -> 5446 -> 5444 -> 5447 -> 5439 -> 5435 -> 5433 -> 5401 -> 5400 -> 5394 -> 5291 -> 5278 -> 5289 -> 5281 -> 5269 -> 5257 -

In [ ]:
import json
import math
import queue as que
import time

with open("./dict_files/Coord.json") as cord:
    coords = json.load(cord)
with open("./dict_files/Cost.json") as Cost:
    cost = json.load(Cost)
with open("./dict_files/Dist.json") as Dist:
    dist = json.load(Dist)
with open("./dict_files/G.json") as G:
    graph = json.load(G)

# Returns euclidean distance between the two nodes that are passed as argmunets to this function
def euc_distance(node, connected_node):
    node_x, node_y = coords[node]
    connected_node_x, connected_node_y = coords[connected_node]
    return math.sqrt((node_x - connected_node_x) ** 2 + (node_y - connected_node_y) ** 2)

# This function manages a dictionary for all the visited nodes' paths
# If any node is visited then it's path is saved as an object in the dictionary, and this
# function will return the cost to reach that node along with the entire path

def append_to_path(path, node, parent):
    # Base case, if the node is empty then only one node is appended to the path and cost 0 is returned
    # as their is only one node in the path
    if len(path) == 0 and parent is None:
        return [node], 0

    # If the path already has one or more nodes in it, then the current node is appended into the parent's path
    # and the resulting path is allocated to the current node. The accumulated cost to reach the current node
    # is also returned
    path_to_return = []
    if parent in path:
        travel_cost = 0
        index = 0
        for path_node in path:
            # If we reach the parent node in path, then we append current node to it and the resulting path to the
            # current node is returned along with it's accumulated cost
            if path_node == parent:
                path_to_return.append(parent)
                travel_cost += dist[path[index]+","+node]
                path_to_return.append(node)
                return path_to_return, travel_cost
            path_to_return.append(path_node)
            travel_cost += dist[(path[index]+","+path[index + 1])]
            index += 1

# This function displays the path as specified in the requirements of any path_list that is passed to it as argument
def display_path(path_list):
    for i in range(len(path_list)):
        if i == len(path_list) - 1:
            print(path_list[i])
            break
        print(path_list[i]+"->", end="")

# Implementation of Greedy Best First Search for task1
def UCS(start_node, goal_node):
    # PriorityQueue is maintained for sorting the accumulated cost for the current node
    priorityQueue = que.PriorityQueue(0)
    priorityQueue.put((0, start_node, None, [start_node]))
    visited_nodes = []
    # Total path is a dictionary that maintains the path_list for all the visited nodes
    total_path = []
    total_dist = 0
    min_dist = 1000000
    iterate = 0
    while not priorityQueue.empty():

        # print("QUEUE: ", priorityQueue)
        total_dist, current_node, parent_node, path = priorityQueue.get()
        # print("Dist: ", euc_dist, "Current_Node: ", current_node, "Parend_Node: ", parent_node)

        # In exception to the first iteration, the current node is appended to the path and resulting cost is
        # provided
        # if parent_node is not None:
        #     path_list = total_path[parent_node]
        #     path_list, total_cost = append_to_path(path_list, current_node, parent_node)
        #     # print("Path: so far: ", path_list)
        #     total_path[current_node] = path_list

        if current_node not in visited_nodes:
            # print("Current Node is not visited")
            visited_nodes.append(current_node)
            if current_node == goal_node:
                if min_dist >= total_dist:
                    total_path = list(path)
                    min_dist = total_dist
                visited_nodes.remove(current_node)
                print("Goal state reached but algo is still finding best solution")
            else:
                neighbors = graph[current_node]

                for neighbor in neighbors:
                    # For GBFS, the approximate euc_dist to reach the goal_node from neighbor is stored in the priority
                    # queue and thus based on shortest euc_dist, the algo works
                    # neighbor_euc_dist = euc_distance(neighbor, goal_node)
                    # print("Neighbor: ", neighbor)
                    # print("dist so far: ", total_cost, "Dist between c node and neighbor: ", dist[(current_node+","+neighbor)])
                    # print("Total dist. till neighbor: ",total_cost + dist[(current_node+","+neighbor)])
                    neighbor_path = list(path)
                    neighbor_path.append(neighbor)
                    # if neighbor not in visited_nodes or neighbor == goal_node:
                    priorityQueue.put((total_dist + dist[current_node+","+neighbor], neighbor, current_node, neighbor_path))

    print("Shortest path: ", end="")
    display_path(total_path)
    print("Shortest distance: ", min_dist)
    print("Total energy cost: ", calculate_cost(total_path))

# This function simply calculates the entire accumulated energy cost for any path list provided to it
def calculate_cost(path):
    path_cost = 0
    for index in range(len(path) - 1):
        path_cost += cost[(path[index] + "," + path[index + 1])]
    return path_cost

# This function simply calculates the entire accumulated distance for any path list provided to it
def calculate_dist(path):
    path_cost = 0
    for index in range(len(path) - 1):
        path_cost += dist[(path[index] + "," + path[index + 1])]
    return path_cost

# For task 2, implementation of Uniform Cost Search
def UCS_energy(start_node, goal_node, energy_budget = 287932):
    # PriorityQueue is maintained for sorting the accumulated cost for the current node
    priorityQueue = que.PriorityQueue()
    priorityQueue.put((0, start_node, None))
    visited_nodes = []
    # Total path is a dictionary that maintains the path_list for all the visited nodes
    total_path = {"1": ["1"]}
    total_cost = 0

    while not priorityQueue.empty():
        travel_cost, current_node, parent_node = priorityQueue.get()

        # In exception to the first iteration, the current node is appended to the path and resulting cost is
        # provided
        if parent_node is not None:
            path_list = total_path[parent_node]
            path_list, total_cost = append_to_path(path_list, current_node, parent_node)
            total_path[current_node] = path_list

        visited_nodes.append(current_node)

        if current_node == goal_node:
            if calculate_cost(total_path[current_node]) > energy_budget:
                visited_nodes.remove(current_node)
            else:
                print("Shortest path: ", end="")
                display_path(total_path[current_node])
                print("Shortest distance: ", calculate_dist(total_path[current_node]))
                print("Total energy cost: ", calculate_cost(total_path[current_node]))
                return

        neighbors = graph[current_node]

        for neighbor in neighbors:
            # For UCS, the accumulated cost from start_node till neighbor node is stored in the priority
            # queue and thus based on shortest euc_dist, the algo works
            if neighbor not in visited_nodes or neighbor == goal_node:
                neighbor_dist = cost[current_node+","+neighbor]
                priorityQueue.put((neighbor_dist + travel_cost, neighbor, current_node))

def Astar(start_node, goal_node, energy_budget = 287932):
    # PriorityQueue is maintained for sorting the accumulated cost for the current node
    priorityQueue = que.PriorityQueue()
    priorityQueue.put((0, start_node, None))
    visited_nodes = []
    # Total path is a dictionary that maintains the path_list for all the visited nodes
    total_path = {"1": ["1"]}
    total_cost = 0

    while not priorityQueue.empty():

        travel_cost, current_node, parent_node = priorityQueue.get()
        dist_to_reach_current_node = 0

        # In exception to the first iteration, the current node is appended to the path and resulting cost is
        # provided
        if parent_node is not None:
            path_list = total_path[parent_node]
            path_list, dist_to_reach_current_node = append_to_path(path_list, current_node, parent_node)
            total_path[current_node] = path_list

        if current_node not in visited_nodes:
            visited_nodes.append(current_node)
            if current_node == goal_node:
                if calculate_cost(total_path[current_node]) > energy_budget:
                    visited_nodes.remove(current_node)
                else:
                    print("Shortest path: ", end="")
                    display_path(total_path[current_node])
                    print("Shortest distance: ", calculate_dist(total_path[current_node]))
                    print("Total energy cost: ", calculate_cost(total_path[current_node]))
                    return

            neighbors = graph[current_node]

            for neighbor in neighbors:
                # For Astar, the accumulated cost from start_node till neighbor node along with the estimated euc_dist
                # to reach the goal node from the neighbor is stored in the priority queue and thus based on shortest
                # euc_dist, the algo works
                neighbor_dist = cost[current_node+","+neighbor]
                dist_to_reach_neighbor = dist_to_reach_current_node + neighbor_dist
                euc_dist_to_goal_node = euc_distance(neighbor, goal_node)
                priorityQueue.put((dist_to_reach_neighbor + euc_dist_to_goal_node, neighbor, current_node))




if __name__ == '__main__':

    path_list = ["1" ,"1363", "1358","1357","1356","1276","1273","1277","1269","1267","1268","1284","1283","1282","1255","1253","1260","1259","1249","1246","963","964","962","1002","952","1000","998","994","995","996","987","988","979","980","969","977","989","990","991","2369","2366","2340","2338","2339","2333","2334","2329","2029","2027","2019","2022","2000","1996","1997","1993","1992","1989","1984","2001","1900","1875","1874","1965","1963","1964","1923","1944","1945","1938","1937","1939","1935","1931","1934","1673","1675","1674","1837","1671","1828","1825","1817","1815","1634","1814","1813","1632","1631","1742","1741","1740","1739","1591","1689","1585","1584","1688","1579","1679","1677","104","5680","5418","5431","5425","5424","5422","5413","5412","5411","66","5392","5391","5388","5291","5278","5289","5290","5283", "5284", "5280", "50"]
    print(calculate_dist(path_list))
    print(calculate_cost(path_list))
    print(graph["50"])


    UCS("1", "50")
    # UCS("1", "50", 287932)
    # Astar("1", "50", 287932)

148648.63722140007
294853
['5268', '5280', '49']
Goal state reached but algo is still finding best solution
Goal state reached but algo is still finding best solution
Goal state reached but algo is still finding best solution
